<a href="https://colab.research.google.com/github/hopebrooke/3931/blob/main/updatedDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load google drive content
#from google.colab import drive
#drive.mount('/content/drive')

# view files in DeePore folder
#%cd /content/drive/MyDrive/hb-final-project/DeePore
#!ls

import DeePore as dp
import numpy as np
import h5py
import matplotlib.pyplot as plt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/hb-final-project/DeePore
Data	    images	Model5.h5    sample_gt.txt	     Tested_Data_Model7.mat
DeePore.py  Logs	Model6.h5    Tested_Data_Model1.mat  Tested_Data_Model8.mat
Demo1.py    minmax.npy	Model7.h5    Tested_Data_Model2.mat  Tested_Data_Model9.mat
Demo2.py    Model1.h5	Model8.h5    Tested_Data_Model3.mat  VarNames.txt
Demo3.py    Model2.h5	Model9.h5    Tested_Data_Model4.mat
Demo4.py    Model3.h5	__pycache__  Tested_Data_Model5.mat
Demo5.py    Model4.h5	results.txt  Tested_Data_Model6.mat


In [ ]:
# dp.show_feature_maps doesn't work on ipynb due to floats/int conversion
# redefining here with same code but added casting
def show_feature_maps(A):
    N= int(np.ceil(np.sqrt(A.shape[0])))
    f=plt.figure(figsize=(N*10,N*10))
    for I in range(A.shape[0]):
        plt.subplot(N,N,I+1)
        plt.imshow(dp.normal(np.squeeze(A[I,:,:,:])))
        plt.axis('off')
    plt.show()

    f.savefig('images/initial_feature_maps.png')

# dp.showentry doesn't work on ipynb due to floats/int conversion
# redefining here with same code but added casting
def showentry(A):
    """shows 3 slices of a volume data """
    A=np.squeeze(A)
    plt.figure(num=None, figsize=(20, 8), dpi=80, facecolor='w', edgecolor='k')
    # CM=plt.cm.jet
    # CM=plt.cm.plasma
    CM=plt.cm.viridis
    ax1=plt.subplot(1,3,1); plt.axis('off'); ax1.set_title('X mid-slice')
    plt.imshow(np.squeeze(A[int(A.shape[0]/2), :,:]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax2=plt.subplot(1,3,2); plt.axis('off'); ax2.set_title('Y mid-slice')
    plt.imshow(np.squeeze(A[:,int(A.shape[1]/2), :]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax3=plt.subplot(1,3,3); plt.axis('off'); ax3.set_title('Z mid-slice');
    plt.imshow(np.squeeze(A[:,:,int(A.shape[2]/2)]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    plt.savefig('images/First_entry.png')



# using monkey patching to update orginal function (not usually advised!)
dp.show_feature_maps = show_feature_maps
dp.showentry = showentry

In [ ]:

# function that takes 2 slices from each direction
def slicevol_2(A):
    A=np.squeeze(A)
    B=np.zeros((1,A.shape[0],A.shape[1],6))
    # slices from front
    B[0,:,:,0]=A[int(A.shape[0]/3),:,:]
    B[0,:,:,1]=A[int(2 * A.shape[0]/3),:,:]
    # slices from left
    B[0,:,:,2]=A[:,int(A.shape[1]/3),:]
    B[0,:,:,3]=A[:,int(2 * A.shape[1]/3),:]
    # slices from top
    B[0,:,:,4]=A[:,:,int(A.shape[2]/3)]
    B[0,:,:,5]=A[:,:,int( 2 * A.shape[2]/3)]
    return B

# function that takes 3 slices from each direction
def slicevol_3(A):
    A=np.squeeze(A)
    B=np.zeros((1,A.shape[0],A.shape[1],9))
    # slices from front
    B[0,:,:,0]=A[int(A.shape[0]/4),:,:]
    B[0,:,:,1]=A[int(2 * A.shape[0]/4),:,:]
    B[0,:,:,2]=A[int(3 * A.shape[0]/4),:,:]
    # slices from left
    B[0,:,:,3]=A[:,int(A.shape[1]/4),:]
    B[0,:,:,4]=A[:,int(2 * A.shape[1]/4),:]
    B[0,:,:,5]=A[:,int(3 * A.shape[1]/4),:]
    # slices from top
    B[0,:,:,6]=A[:,:,int(A.shape[2]/4)]
    B[0,:,:,7]=A[:,:,int( 2 * A.shape[2]/4)]
    B[0,:,:,8]=A[:,:,int( 3 * A.shape[2]/4)]
    return B

In [ ]:
# if we change slice volume, we change how the compact dataset is created...
# same as dp.create_compact_dataset but changing parameters for writeh5slice() call

# for 2 slice in each direction:
def create_compact_dataset_2(Path_complete,Path_compact):
    S=dp.hdf_shapes(Path_complete,['X'])
    for I in range(S[0][0]):
        X=dp.readh5slice(Path_complete,'X',[I])
        Y=dp.readh5slice(Path_complete,'Y',[I])
        X=dp.slicevol(X)
        X=dp.ecl_distance(X)
        dp.writeh5slice(X,Path_compact,'X',Shape=[128,128,6])
        dp.writeh5slice(Y,Path_compact,'Y',Shape=[1515,1])

# for 3 slice in each direction
def create_compact_dataset_3(Path_complete,Path_compact):
    S=dp.hdf_shapes(Path_complete,['X'])
    for I in range(S[0][0]):
        X=dp.readh5slice(Path_complete,'X',[I])
        Y=dp.readh5slice(Path_complete,'Y',[I])
        X=dp.slicevol(X)
        X=dp.ecl_distance(X)
        dp.writeh5slice(X,Path_compact,'X',Shape=[128,128,9])
        dp.writeh5slice(Y,Path_compact,'Y',Shape=[1515,1])

In [ ]:
# changing slice vol also means we need to change how the entries are shown
# same as dp.showentry


def showentry_2(A):
    """shows 6 slices of a volume data """
    A=np.squeeze(A)
    plt.figure(num=None, figsize=(20, 14), dpi=80, facecolor='w', edgecolor='k')
    CM=plt.cm.viridis

    # X slices
    ax1=plt.subplot(2,3,1); plt.axis('off'); ax1.set_title('X mid-slice 1')
    plt.imshow(np.squeeze(A[int(A.shape[0]/3), :,:]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax2=plt.subplot(2,3,4); plt.axis('off'); ax2.set_title('X mid-slice 2')
    plt.imshow(np.squeeze(A[int(2 * A.shape[0]/3), :,:]), cmap=CM, interpolation='nearest')

    # Y slices
    ax3=plt.subplot(2,3,2); plt.axis('off'); ax3.set_title('Y mid-slice 1')
    plt.imshow(np.squeeze(A[:,int(A.shape[1]/3), :]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax4=plt.subplot(2,3,5); plt.axis('off'); ax4.set_title('Y mid-slice 2')
    plt.imshow(np.squeeze(A[:,int(2 * A.shape[1]/3), :]), cmap=CM, interpolation='nearest')

    # Z slices
    ax5=plt.subplot(2,3,3); plt.axis('off'); ax5.set_title('Z mid-slice 1');
    plt.imshow(np.squeeze(A[:,:,int(A.shape[2]/3)]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax6=plt.subplot(2,3,6); plt.axis('off'); ax6.set_title('Z mid-slice 2');
    plt.imshow(np.squeeze(A[:,:,int(2 * A.shape[2]/3)]), cmap=CM, interpolation='nearest')

    plt.savefig('images/First_entry_2.png')


def showentry_3(A):
    """shows 9 slices of a volume data """
    A=np.squeeze(A)
    plt.figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
    CM=plt.cm.viridis

    # X slices
    ax1=plt.subplot(3,3,1); plt.axis('off'); ax1.set_title('X mid-slice 1')
    plt.imshow(np.squeeze(A[int(A.shape[0]/4), :,:]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax2=plt.subplot(3,3,4); plt.axis('off'); ax2.set_title('X mid-slice 2')
    plt.imshow(np.squeeze(A[int(2 * A.shape[0]/4), :,:]), cmap=CM, interpolation='nearest')
    ax3=plt.subplot(3,3,7); plt.axis('off'); ax3.set_title('X mid-slice 3')
    plt.imshow(np.squeeze(A[int(3 * A.shape[0]/4), :,:]), cmap=CM, interpolation='nearest')

    # Y slices
    ax4=plt.subplot(3,3,2); plt.axis('off'); ax4.set_title('Y mid-slice 1')
    plt.imshow(np.squeeze(A[:,int(A.shape[1]/4), :]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax5=plt.subplot(3,3,5); plt.axis('off'); ax5.set_title('Y mid-slice 2')
    plt.imshow(np.squeeze(A[:,int(2 * A.shape[1]/4), :]), cmap=CM, interpolation='nearest')
    ax6=plt.subplot(3,3,8); plt.axis('off'); ax6.set_title('Y mid-slice 3')
    plt.imshow(np.squeeze(A[:,int(3 * A.shape[1]/4), :]), cmap=CM, interpolation='nearest')

    # Z slices
    ax7=plt.subplot(3,3,3); plt.axis('off'); ax7.set_title('Z mid-slice 1');
    plt.imshow(np.squeeze(A[:,:,int(A.shape[2]/4)]), cmap=CM, interpolation='nearest')
    # plt.colorbar(orientation="horizontal")
    ax8=plt.subplot(3,3,6); plt.axis('off'); ax8.set_title('Z mid-slice 2');
    plt.imshow(np.squeeze(A[:,:,int(2 * A.shape[2]/4)]), cmap=CM, interpolation='nearest')
    ax9=plt.subplot(3,3,9); plt.axis('off'); ax9.set_title('Z mid-slice 3');
    plt.imshow(np.squeeze(A[:,:,int(3 * A.shape[2]/4)]), cmap=CM, interpolation='nearest')

    plt.savefig('images/First_entry_3.png')
